In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_excel('sample.xlsx', sheet_name='一级考核-错号率')
raw = df.copy(deep=True)
raw.head()

,行标签,S2题甄别失败,S4a题甄别失败,S4a甄别失败,S4b题甄别失败,S4b甄别失败,S6a题甄别失败,S6a甄别失败,S6题甄别失败,S6甄别失败,S7题甄别失败,S7甄别失败,成功,关机/占线/无法接通/语音信箱,过滤问卷拒访,开场白拒访,空号/号码过期,磨合期-延期交付（未交付）,内部员工,审核作废,时间不宜,剔除名单,剔除-无电话号码,剔除-重复名单,停机,停机/空号/号码过期,未购买此楼盘,未接触,无此人/打错了,无人接听,语言不通,直接挂断/被访者不说话,重复访问,主体问卷拒访,准业主-提前交付（已交付）,作废,总计,错号数,接通数,错号率
0,集团,355.0,362.0,28.0,373.0,15.0,2201.0,12.0,217.0,8.0,1087.0,16.0,78199.0,63287.0,1379.0,28414.0,3247.0,13.0,251.0,27.0,27544.0,3.0,1.0,1.0,1554.0,2580.0,838.0,120011.0,3091.0,150889.0,1045.0,11697.0,1218.0,8263.0,103.0,1.0,388319,11665,174143,0.030040
1,上海区域,43.0,47.0,2.0,31.0,6.0,213.0,2.0,55.0,4.0,100.0,6.0,10440.0,8893.0,153.0,2891.0,364.0,1.0,40.0,13.0,3218.0,1.0,NaN,NaN,225.0,533.0,176.0,16097.0,571.0,21313.0,77.0,1212.0,76.0,1036.0,23.0,1.0,51766,1912,21560,0.036935
2,常徐,10.0,9.0,NaN,7.0,2.0,23.0,NaN,2.0,NaN,7.0,NaN,1642.0,1850.0,23.0,488.0,38.0,NaN,3.0,NaN,479.0,NaN,NaN,NaN,33.0,86.0,26.0,2524.0,82.0,3220.0,17.0,301.0,13.0,156.0,4.0,NaN,8521,275,3451,0.032273
3,1-准业主1,3.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,663.0,538.0,6.0,156.0,11.0,NaN,1.0,NaN,195.0,NaN,NaN,NaN,7.0,33.0,9.0,548.0,28.0,1230.0,3.0,75.0,5.0,71.0,1.0,NaN,3044,91,1276,0.029895
4,2-准业主2,2.0,NaN,NaN,7.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,487.0,607.0,4.0,128.0,8.0,NaN,2.0,NaN,157.0,NaN,NaN,NaN,16.0,47.0,6.0,743.0,28.0,960.0,8.0,54.0,2.0,42.0,3.0,NaN,2570,107,1003,0.041634


In [3]:
def simplify(raw: pd.DataFrame):    
    labels = ['行标签', '总计', '接触量', '错号数', '错号量']
    for name in raw.columns.tolist():
        if name not in labels:
            raw.drop(name, axis=1, inplace=True) 
simplify(raw)
raw.head(10)

,行标签,总计,错号数
0,集团,388319,11665
1,上海区域,51766,1912
2,常徐,8521,275
3,1-准业主1,3044,91
4,2-准业主2,2570,107
5,3-磨合期1,379,10
6,4-磨合期2,568,19
7,5-稳定期,270,8
8,6-老业主,1690,40
9,南京,6303,139


In [4]:
raw.to_excel('sampleTest.xlsx', index=False)

In [5]:
city_names = []
deprecated_names = ['集团', '上海区域']
for name in raw['行标签'].tolist():
    if name not in deprecated_names and '准业主' not in name \
                                    and '磨合期' not in name \
                                    and '老业主' not in name \
                                    and '稳定期' not in name:
        city_names.append(name)

print(city_names)
len(city_names)

['常徐', '南京', '南通', '内蒙', '宁夏A', '浦东', '浦西', '润扬', '商置', '苏州', '无锡', '无锡文旅', '新疆']


13

In [6]:
row_records = []
for _, row in raw.iterrows():
    row_records.append([row['行标签'], row['总计'], row['错号数']])
    # print(row['行标签'], row['总计'], row['错号数'])
row_records

[['集团', 388319, 11665],
 ['上海区域', 51766, 1912],
 ['常徐', 8521, 275],
 ['1-准业主1', 3044, 91],
 ['2-准业主2', 2570, 107],
 ['3-磨合期1', 379, 10],
 ['4-磨合期2', 568, 19],
 ['5-稳定期', 270, 8],
 ['6-老业主', 1690, 40],
 ['南京', 6303, 139],
 ['1-准业主1', 1773, 33],
 ['2-准业主2', 1577, 28],
 ['3-磨合期1', 1276, 36],
 ['4-磨合期2', 707, 25],
 ['5-稳定期', 952, 17],
 ['6-老业主', 18, 0],
 ['南通', 2223, 100],
 ['1-准业主1', 915, 11],
 ['2-准业主2', 1131, 86],
 ['3-磨合期1', 114, 3],
 ['4-磨合期2', 63, 0],
 ['内蒙', 169, 2],
 ['1-准业主1', 169, 2],
 ['宁夏A', 59, 1],
 ['1-准业主1', 59, 1],
 ['浦东', 186, 3],
 ['1-准业主1', 186, 3],
 ['浦西', 2191, 106],
 ['1-准业主1', 1139, 55],
 ['2-准业主2', 704, 30],
 ['3-磨合期1', 234, 18],
 ['4-磨合期2', 114, 3],
 ['润扬', 2255, 32],
 ['1-准业主1', 1078, 13],
 ['2-准业主2', 830, 15],
 ['3-磨合期1', 347, 4],
 ['商置', 245, 5],
 ['1-准业主1', 1, 0],
 ['3-磨合期1', 119, 2],
 ['4-磨合期2', 125, 3],
 ['苏州', 6751, 370],
 ['1-准业主1', 2602, 139],
 ['2-准业主2', 1251, 80],
 ['3-磨合期1', 1212, 92],
 ['4-磨合期2', 451, 31],
 ['5-稳定期', 651, 13],
 ['6-老业主', 584, 15],
 ['无

In [7]:
wrong2city = {}
city_appeared = []
cities = []
slice_idx = []
zyz = []
mhq = []
wdq = []
lyz = []

for i in range(2, len(row_records)):
    tag = row_records[i][0]
    if tag in city_names:
        wrong2city[row_records[i][2]] = tag
        city_appeared.append(tag)
        slice_idx.append(i)
slice_idx.append(len(row_records))
for i in range(len(slice_idx)-1):
    cities.append(row_records[slice_idx[i]:slice_idx[i+1]])
print(cities[0])    
print(city_appeared)
print(slice_idx)

[['常徐', 8521, 275], ['1-准业主1', 3044, 91], ['2-准业主2', 2570, 107], ['3-磨合期1', 379, 10], ['4-磨合期2', 568, 19], ['5-稳定期', 270, 8], ['6-老业主', 1690, 40]]
['常徐', '南京', '南通', '内蒙', '宁夏A', '浦东', '浦西', '润扬', '商置', '苏州', '无锡', '无锡文旅', '新疆']
[2, 9, 16, 21, 23, 25, 27, 32, 36, 40, 47, 54, 57, 61]


In [8]:
for city_info in cities:
    CITY = city_info[0][0]
    for line in city_info[1:]:
        if '准业主' in line[0]:
            line[0] = CITY + '-' + line[0]
            zyz.append(line)
        if '磨合期' in line[0]:
            line[0] = CITY + '-' + line[0]
            mhq.append(line)
        if '稳定期' in line[0]:
            line[0] = CITY + '-' + line[0]
            wdq.append(line)
        if '老业主' in line[0]:
            line[0] = CITY + '-' + line[0]
            lyz.append(line)
zyz

[['常徐-1-准业主1', 3044, 91],
 ['常徐-2-准业主2', 2570, 107],
 ['南京-1-准业主1', 1773, 33],
 ['南京-2-准业主2', 1577, 28],
 ['南通-1-准业主1', 915, 11],
 ['南通-2-准业主2', 1131, 86],
 ['内蒙-1-准业主1', 169, 2],
 ['宁夏A-1-准业主1', 59, 1],
 ['浦东-1-准业主1', 186, 3],
 ['浦西-1-准业主1', 1139, 55],
 ['浦西-2-准业主2', 704, 30],
 ['润扬-1-准业主1', 1078, 13],
 ['润扬-2-准业主2', 830, 15],
 ['商置-1-准业主1', 1, 0],
 ['苏州-1-准业主1', 2602, 139],
 ['苏州-2-准业主2', 1251, 80],
 ['无锡-1-准业主1', 4777, 113],
 ['无锡-2-准业主2', 2552, 82],
 ['无锡文旅-1-准业主1', 283, 7],
 ['无锡文旅-2-准业主2', 1, 0],
 ['新疆-1-准业主1', 348, 8],
 ['新疆-2-准业主2', 120, 2]]

In [9]:
def analyze(l: list):
    res = {}
    for i in range(len(l)):
        city = l[i][0].split('-')[0]
        if not res.__contains__(city):
            res[city] = [l[i][1], l[i][2]]
        else:
            res[city][0] += l[i][1]
            res[city][1] += l[i][2]
    return res
print(analyze(zyz))
print(analyze(mhq))
print(analyze(wdq))
print(analyze(lyz))

{'常徐': [5614, 198], '南京': [3350, 61], '南通': [2046, 97], '内蒙': [169, 2], '宁夏A': [59, 1], '浦东': [186, 3], '浦西': [1843, 85], '润扬': [1908, 28], '商置': [1, 0], '苏州': [3853, 219], '无锡': [7329, 195], '无锡文旅': [284, 7], '新疆': [468, 10]}
{'常徐': [947, 29], '南京': [1983, 61], '南通': [177, 3], '浦西': [348, 21], '润扬': [347, 4], '商置': [244, 5], '苏州': [1663, 123], '无锡': [7717, 544], '新疆': [4, 0]}
{'常徐': [270, 8], '南京': [952, 17], '苏州': [651, 13], '无锡': [214, 3]}
{'常徐': [1690, 40], '南京': [18, 0], '苏州': [584, 15], '无锡': [6847, 120]}


In [10]:
def calculate(d: dict):
    for k in d.keys():
        print(k, d[k])
        d[k].append(float('{:.5f}'.format(d[k][1]/d[k][0])))
    return d
print(calculate(analyze(zyz)))
print(calculate(analyze(mhq)))
print(calculate(analyze(wdq)))
print(calculate(analyze(lyz)))

常徐 [5614, 198]
南京 [3350, 61]
南通 [2046, 97]
内蒙 [169, 2]
宁夏A [59, 1]
浦东 [186, 3]
浦西 [1843, 85]
润扬 [1908, 28]
商置 [1, 0]
苏州 [3853, 219]
无锡 [7329, 195]
无锡文旅 [284, 7]
新疆 [468, 10]
{'常徐': [5614, 198, 0.03527], '南京': [3350, 61, 0.01821], '南通': [2046, 97, 0.04741], '内蒙': [169, 2, 0.01183], '宁夏A': [59, 1, 0.01695], '浦东': [186, 3, 0.01613], '浦西': [1843, 85, 0.04612], '润扬': [1908, 28, 0.01468], '商置': [1, 0, 0.0], '苏州': [3853, 219, 0.05684], '无锡': [7329, 195, 0.02661], '无锡文旅': [284, 7, 0.02465], '新疆': [468, 10, 0.02137]}
常徐 [947, 29]
南京 [1983, 61]
南通 [177, 3]
浦西 [348, 21]
润扬 [347, 4]
商置 [244, 5]
苏州 [1663, 123]
无锡 [7717, 544]
新疆 [4, 0]
{'常徐': [947, 29, 0.03062], '南京': [1983, 61, 0.03076], '南通': [177, 3, 0.01695], '浦西': [348, 21, 0.06034], '润扬': [347, 4, 0.01153], '商置': [244, 5, 0.02049], '苏州': [1663, 123, 0.07396], '无锡': [7717, 544, 0.07049], '新疆': [4, 0, 0.0]}
常徐 [270, 8]
南京 [952, 17]
苏州 [651, 13]
无锡 [214, 3]
{'常徐': [270, 8, 0.02963], '南京': [952, 17, 0.01786], '苏州': [651, 13, 0.01997], '无锡': [214, 

In [11]:
zyz_list = []
mhq_list = []
wdq_list = []
lyz_list = []
for k, v in calculate(analyze(zyz)).items():
    zyz_list.append(tuple([k, '准业主'] + v))
for k, v in calculate(analyze(mhq)).items():
    mhq_list.append(tuple([k, '磨合期'] + v))
for k, v in calculate(analyze(wdq)).items():
    wdq_list.append(tuple([k, '稳定期'] + v))
for k, v in calculate(analyze(lyz)).items():
    lyz_list.append(tuple([k, '老业主'] + v))

常徐 [5614, 198]
南京 [3350, 61]
南通 [2046, 97]
内蒙 [169, 2]
宁夏A [59, 1]
浦东 [186, 3]
浦西 [1843, 85]
润扬 [1908, 28]
商置 [1, 0]
苏州 [3853, 219]
无锡 [7329, 195]
无锡文旅 [284, 7]
新疆 [468, 10]
常徐 [947, 29]
南京 [1983, 61]
南通 [177, 3]
浦西 [348, 21]
润扬 [347, 4]
商置 [244, 5]
苏州 [1663, 123]
无锡 [7717, 544]
新疆 [4, 0]
常徐 [270, 8]
南京 [952, 17]
苏州 [651, 13]
无锡 [214, 3]
常徐 [1690, 40]
南京 [18, 0]
苏州 [584, 15]
无锡 [6847, 120]


In [12]:
labels = ['城市', '业主阶段', '接触量', '错号数', '错号率']
zyz_xlxs = pd.DataFrame(data=zyz_list, columns=labels)
zyz_xlxs = zyz_xlxs.sort_values(by='错号率', ascending=False)
zyz_xlxs.head(20)

,城市,业主阶段,接触量,错号数,错号率
9,苏州,准业主,3853,219,0.05684
2,南通,准业主,2046,97,0.04741
6,浦西,准业主,1843,85,0.04612
0,常徐,准业主,5614,198,0.03527
10,无锡,准业主,7329,195,0.02661
11,无锡文旅,准业主,284,7,0.02465
12,新疆,准业主,468,10,0.02137
1,南京,准业主,3350,61,0.01821
4,宁夏A,准业主,59,1,0.01695
5,浦东,准业主,186,3,0.01613


In [13]:
mhq_xlxs = pd.DataFrame(data=mhq_list, columns=labels)
mhq_xlxs = mhq_xlxs.sort_values(by='错号率', ascending=False)
mhq_xlxs.head(20)

,城市,业主阶段,接触量,错号数,错号率
6,苏州,磨合期,1663,123,0.07396
7,无锡,磨合期,7717,544,0.07049
3,浦西,磨合期,348,21,0.06034
1,南京,磨合期,1983,61,0.03076
0,常徐,磨合期,947,29,0.03062
5,商置,磨合期,244,5,0.02049
2,南通,磨合期,177,3,0.01695
4,润扬,磨合期,347,4,0.01153
8,新疆,磨合期,4,0,0.00000


In [14]:
wdq_xlxs = pd.DataFrame(data=wdq_list, columns=labels)
wdq_xlxs = wdq_xlxs.sort_values(by='错号率', ascending=False)
wdq_xlxs.head(20)

,城市,业主阶段,接触量,错号数,错号率
0,常徐,稳定期,270,8,0.02963
2,苏州,稳定期,651,13,0.01997
1,南京,稳定期,952,17,0.01786
3,无锡,稳定期,214,3,0.01402


In [15]:
lyz_xlxs = pd.DataFrame(data=lyz_list, columns=labels)
lyz_xlxs = lyz_xlxs.sort_values(by='错号率', ascending=False)
lyz_xlxs.head(20)

,城市,业主阶段,接触量,错号数,错号率
2,苏州,老业主,584,15,0.02568
0,常徐,老业主,1690,40,0.02367
3,无锡,老业主,6847,120,0.01753
1,南京,老业主,18,0,0.00000


In [16]:
# Shanghai
df = pd.read_excel('sample.xlsx', sheet_name='上海区域')
raw = df.copy(deep=True)
raw.head()
simplify(raw)
raw.head(5)

,行标签,总计,接触量,错号量
0,上海区域,67863,51766,1912
1,准业主,31926,27110,906
2,磨合期,19787,13430,790
3,稳定期,3006,2087,41
4,老业主,13144,9139,175


In [17]:
for _, rows in raw.iterrows():
    if '准业主' in rows['行标签']:
        # tuple = ('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量'])))
        # print(('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
        shzyz = pd.DataFrame({'城市': ['上海区域'],
                             '业主阶段': ['准业主'],
                             '接触量': rows['接触量'],
                             '错号数': rows['错号量'],
                             '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
        zyz_xlxs = pd.concat([zyz_xlxs, shzyz])

    if '磨合期' in rows['行标签']:
        print(('上海区域', 'mhq', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
        shmhq = pd.DataFrame({'城市': ['上海区域'],
                             '业主阶段': ['磨合期'],
                             '接触量': rows['接触量'],
                             '错号数': rows['错号量'],
                             '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
        
        mhq_xlxs = pd.concat([mhq_xlxs, shmhq])
    if '稳定期' in rows['行标签']:
        print(('上海区域', 'wdq', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
        shwdq = pd.DataFrame({'城市': ['上海区域'],
                             '业主阶段': ['稳定期'],
                             '接触量': rows['接触量'],
                             '错号数': rows['错号量'],
                             '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
        wdq_xlxs = pd.concat([wdq_xlxs, shwdq])
    if '老业主' in rows['行标签']:
        print(('上海区域', 'lyz', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
        shlyz = pd.DataFrame({'城市': ['上海区域'],
                             '业主阶段': ['老业主'],
                             '接触量': rows['接触量'],
                             '错号数': rows['错号量'],
                             '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
        lyz_xlxs = pd.concat([lyz_xlxs, shlyz])

zyz_xlxs.head(30)

('上海区域', 'mhq', 13430, 790, 0.05882)
('上海区域', 'wdq', 2087, 41, 0.01965)
('上海区域', 'lyz', 9139, 175, 0.01915)


,城市,业主阶段,接触量,错号数,错号率
9,苏州,准业主,3853,219,0.05684
2,南通,准业主,2046,97,0.04741
6,浦西,准业主,1843,85,0.04612
0,常徐,准业主,5614,198,0.03527
10,无锡,准业主,7329,195,0.02661
11,无锡文旅,准业主,284,7,0.02465
12,新疆,准业主,468,10,0.02137
1,南京,准业主,3350,61,0.01821
4,宁夏A,准业主,59,1,0.01695
5,浦东,准业主,186,3,0.01613


In [18]:
mhq_xlxs.head(20)

,城市,业主阶段,接触量,错号数,错号率
6,苏州,磨合期,1663,123,0.07396
7,无锡,磨合期,7717,544,0.07049
3,浦西,磨合期,348,21,0.06034
1,南京,磨合期,1983,61,0.03076
0,常徐,磨合期,947,29,0.03062
5,商置,磨合期,244,5,0.02049
2,南通,磨合期,177,3,0.01695
4,润扬,磨合期,347,4,0.01153
8,新疆,磨合期,4,0,0.00000
0,上海区域,磨合期,13430,790,0.05882


In [19]:
wdq_xlxs.head(10)

,城市,业主阶段,接触量,错号数,错号率
0,常徐,稳定期,270,8,0.02963
2,苏州,稳定期,651,13,0.01997
1,南京,稳定期,952,17,0.01786
3,无锡,稳定期,214,3,0.01402
0,上海区域,稳定期,2087,41,0.01965


In [20]:
lyz_xlxs.head(10)

,城市,业主阶段,接触量,错号数,错号率
2,苏州,老业主,584,15,0.02568
0,常徐,老业主,1690,40,0.02367
3,无锡,老业主,6847,120,0.01753
1,南京,老业主,18,0,0.00000
0,上海区域,老业主,9139,175,0.01915


In [21]:
# writer = pd.ExcelWriter('sample.xlsx', engine='xlsxwriter')
# workbook = writer.book
# worksheet = workbook.add_worksheet('Auto')
# writer.sheets['Auto'] = worksheet

# zyz_xlxs.to_excel(writer, sheet_name='Auto', startrow=0, startcol=0)
# mhq_xlxs.to_excel(writer, sheet_name='Auto', startrow=0, startcol=7)
# wdq_xlxs.to_excel(writer, sheet_name='Auto', startrow=0, startcol=14)
# lyz_xlxs.to_excel(writer, sheet_name='Auto', startrow=0, startcol=21)

In [22]:
# funtion
def multiple_dfs(df_list, sheets, file_name, spaces):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    row = 0
    for dataframe in df_list:
        dataframe.to_excel(writer,sheet_name=sheets,startrow=row , startcol=0, index=False)   
        row = row + len(dataframe.index) + spaces + 1
    writer.save()

# list of dataframes
dfs = [zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs]

# run function
multiple_dfs(dfs, 'Validation', 'test1.xlsx', 1)